In [2]:
import h5py
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt, decimate
import numpy as np
import datetime
import pandas as pd
import obspy
from obspy import UTCDateTime
from obspy.clients.fdsn import Client
import time
import glob
import os
import pytz
from tqdm import tqdm
import csv
import matplotlib.dates as mdates

In [3]:
#Loading template swarm 2023-08-27
#loading files

file_list = glob.glob("/auto/petasaur-wd15/rainier-10-14-2023-drive1/rainier/*")

file_list.sort()
len(file_list)

36753

In [ ]:
data_file2 = h5py.File('/auto/petasaur-wd15/rainier-10-14-2023-drive1/rainier/decimator_2023-08-27_10.10.00_UTC.h5','r')

In [ ]:
data_file2
chan_min=0
chan_max=-1
this_data2 = np.array(data_file2['Acquisition/Raw[0]/RawData'][:,chan_min:chan_max])
this_time2 = np.array(data_file2['Acquisition/Raw[0]/RawDataTime'])


In [3]:
# Establishing frequencies

original_frequency = 200  # Hz
desire_frequency = 20    # Hz
chan_min=0
chan_max=-1

# butter fuction
orden_fiilter = 4
cutoff_frequency = desire_frequency / 2.0  # Fequency in Hz

#defining filter

b, a = butter(orden_fiilter, cutoff_frequency / (original_frequency / 2.0), btype='low')

# Iteration for file_list
output_folder = "/data/data4/veronica-scratch-rainier/drive1_ds" #+ str(decimator)

total_start_time = time.perf_counter()

for file_name in file_list:
    # loading  HDF5 files
    with h5py.File(file_name, 'r') as file:
        # Obtener el conjunto de datos
        original_data = file['Acquisition/Raw[0]/RawData'][:, chan_min:chan_max]
        original_time = np.array(file['Acquisition/Raw[0]/RawDataTime'])
        attrs = dict(file['Acquisition'].attrs)  # Copy attrs
        
        

    # Calcular el factor de decimación
    factor_decimacion = original_frequency // desire_frequency

    # Aplicar el filtro
    data_filtrada = filtfilt(b, a, original_data, axis=0)

    # Decimar el resultado
    downsampled_data = decimate(data_filtrada, factor_decimacion, axis=0)
    
    # decimation on time time too
    downsampled_time = original_time[:original_time.shape[0]:factor_decimacion]
    
    #newfile name
    output_file_name = f"new_{file_name.split('/')[-1]}"

    #  saving them
    with h5py.File(output_folder + '/' + output_file_name, 'w') as g:
        g.create_dataset('Acquisition/Raw[0]/RawData', data=downsampled_data)
        g.create_dataset('Acquisition/Raw[0]/RawDataTime', data=downsampled_time, dtype='f8')
        g['Acquisition'].attrs.update(attrs)
        g.close()

total_end_time = time.perf_counter()
        
total_elapsed_time = total_end_time - total_start_time
print(f"Tiempo total: {total_elapsed_time} segundos")


Tiempo total: 10337.368205309846 segundos
